In [ ]:
#Initial conditions for the parallaxconst=(moon distance in Earth radii,geocentric RA,geocentric Dec)
parallaxinitialconst=[56.0,8.17,20.44]

#Test apparentSkyPosition
key="RD"
site=sites[key]
#location="Moon Center"
location="L1-21J"
apparentSkyPosition(parallaxinitialconst,site["lat"],site["lon"],site["alt"])

#Test fit parallax
parallaxsq(parallaxinitialconst,sites,"Moon Center",verbose=True)

In [ ]:
#Instrument properties
#LaLoma (San Vicente Ferrer, Colombia)
LALOMA_F=0.73
LALOMA_FL=2700.0 #mm
LALOMA_D=635 #mm
ZWO_COLS=4656.0 #px
ZWO_ROWS=3520.0 #px
ZWO_WIDTH=17.5 #mm
ZWO_HEIGHT=13.4 #mm

In [ ]:
#Theoretical spatial resolution of LaLoma telescope
pxsize=ZWO_WIDTH/(LALOMA_F*LALOMA_FL)/ZWO_COLS
pxsize_km=pxsize*MOON_DISTANCE
#At 70 degrees inclination
pxsize_inc=pxsize_km/np.cos(IMPACT_LOC["lat"]*DEG)
print(f"Theoretical angular resolution = {pxsize*180/np.pi*3600} arcsec")
print(f"Theoretical linear resolution = {pxsize_km} km")
print(f"Theoretical linear resolution (@{np.abs(IMPACT_LOC['lat'])} deg) = {pxsize_inc} km")

In [ ]:
solutionselenographic.x=np.array([-1.95449213e-02,  4.72321047e-03,  8.12963942e+00,  6.81466867e-02,
        2.73041416e-01,  2.02204853e+01, -2.50000867e+00, -7.99953324e-01])
print(selensq(solutionselenographic.x,site,LunarSites,verbose=True))

In [ ]:
def getradec(plateconst,x,y,xcen,ycen):
    """
    Get (RA,Dec) relative to center of frame from (X,Y) given plate constant parameters
    """
    dec = plateconst[3]*(x-xcen)+plateconst[4]*(y-ycen)+plateconst[5]
    ra = 15*math.cos(math.radians(dec))*(plateconst[0]*(x-xcen)-plateconst[1]*(y-ycen))+plateconst[2]
    return ra,dec
                         

def distsq(plateconst,stars=None,verbose=False):
    """
    Calculate distance squared for positions given plate constant and star coordinates
    """
    distsq=0
    drasq=[]
    ddecsq=[]
    center=stars.loc[stars["ObjName"]==REF_LUNARSITE].iloc[0]
    #print(center)
    for ind,star in stars.loc[stars["type"]=="Star"].iterrows():
        #print(star)
        ra,dec=getradec(plateconst,star["X"],star["Y"],center["X"],center["Y"])
        drasq+=[(ra-star["RA"])**2]
        ddecsq+=[(dec-star["DEC"])**2]
        distsq+=225*math.cos(math.radians(dec))*drasq[-1]+ddecsq[-1]
    if verbose:return distsq,drasq,ddecsq
    return distsq

def apparentSkyPosition(parallaxconst,lat,lon,ele,gst=GST):
    """
    Get apparent coordinate (RA,Dec) of observer at (lat,lon,ele) given the 
    parallaxconst=(moon distance in Earth radii,geocentric RA,geocentric Dec)
    """
    LST=gst+lon/15
    HA=(LST-parallaxconst[1])*15
    rhosin=0.996647*math.sin(math.atan(0.996647*math.tan(math.radians(lat))))+ele/6378160*math.sin(math.radians(lat))
    rhocos=math.cos(math.atan(0.996647*math.tan(math.radians(lat))))+ele/6378160*math.cos(math.radians(lat))
    deltara=math.degrees(math.atan(rhocos*math.sin(math.radians(HA))/(parallaxconst[0]*math.cos(math.radians(parallaxconst[2]))-rhocos*math.cos(math.radians(HA)))))
    ra=parallaxconst[1]-deltara/15
    dec=math.degrees(math.atan(math.cos(math.radians(HA+deltara))*((parallaxconst[0]*math.sin(math.radians(parallaxconst[2]))-rhosin)/(parallaxconst[0]*math.cos(math.radians(parallaxconst[2]))*math.cos(math.radians(HA))-rhocos))))
    return ra,dec


def parallaxsq(parallaxconst,sites=None,location=None,verbose=False):
    """
    Calculate Error Squared for each parallax correction
    """
    parallaxsq=0
    prasq=[]
    pdecsq=[]
    for key,site in sites.items():
        objs=site["data"]
        #print(key,objs.loc[objs["ObjName"]==location])
        loc=objs.loc[objs["ObjName"]==location].iloc[0]
        ra,dec=apparentSkyPosition(parallaxconst,site["lat"],site["lon"],site["alt"])
        prasq+=[(ra-loc["RA"])**2]
        pdecsq+=[(dec-loc["DEC"])**2]
        parallaxsq+=225*math.cos(math.radians(dec))*prasq[-1]+pdecsq[-1]
    if verbose: return parallaxsq,prasq,pdecsq
    return parallaxsq

# Get (RA,Dec) given selenographic coordinate (lat,lon)
def selenographictoradec(params,lat,lon):
    print(params,lat,lon)
    ra = params[0]*math.sin(math.radians(lon-params[6]))*math.cos(math.radians(lat-params[7]))+params[1]*math.sin(math.radians(lat-params[7]))+params[2]
    dec = params[3]*math.sin(math.radians(lon-params[6]))*math.cos(math.radians(lat-params[7]))+params[4]*math.sin(math.radians(lat-params[7]))+params[5]
    return ra,dec

# Calculates the error between selenographictoradec conversion and (raref,decref) given params
def selenerr(coord,raref,decref,params):
    ra,dec=selenographictoradec(params,coord[0],coord[1])
    return 225*math.cos(math.radians(dec))*(ra-raref)**2+(dec-decref)**2
    
# Calculate Error Squared for each RA,Dec conversion from selenographic coordinate
def selensq(params,site=None,lunarsites=None,verbose=False):
    selensq=0
    srasq=[]
    sdecsq=[]
    
    objs=site["data"]
    locations=objs.loc[objs["type"]=="Moon"]
    
    for ind in locations.index[2:]:
        location=locations.loc[ind]
        locname=location["ObjName"]
        sra=location["RA"]
        sdec=location["DEC"]
        lunarsite=lunarsites.loc[lunarsites["LunarSite"]==locname].iloc[0]
        if lunarsite["slon"]==-1:continue
        ra,dec=selenographictoradec(params,lunarsite["slat"],lunarsite["slon"])
        srasq+=[(ra-sra)**2]
        sdecsq+=[(dec-sdec)**2]
        selensq+=225*math.cos(math.radians(dec))*srasq[-1]+sdecsq[-1]
    if verbose: return selensq,srasq,sdecsq
    return selensq

In [ ]:
"""
Compute the velocity distribution moments and from there reconstruct the distribution.

Use cubic splines: 

John, V., et al. "Techniques for the reconstruction of a
distribution from a finite number of its moments." Chemical
Engineering Science 62.11 (2007): 2890-2904.

"""
suf="CAMS"
IMPACT_SUFFIX3="lat_3.67602e+01__lon_-1.21499e+02"
data_rays=np.loadtxt(f"{IMPACT_FIGDIR}/rays-{IMPACT_SUFFIX3}-{suf}.data.phys")
data_rays_prob=np.loadtxt(f"{IMPACT_FIGDIR}/rays-{IMPACT_SUFFIX3}-{suf}.data.prob")
Nrays=len(data_rays)
print(f"Number of test trajectories read: {Nrays}")


data=data_rays
dataprob=data_rays_prob

# GET ONLY THE ACCEPTED RAYS
vimps=data[:,2]
pprob=dataprob[:,7]

print("Computed rays:",len(pprob))
print("Acepted rays:",len(vimps))

# NORMALIZATION
#norm=100
norm=300
#norm=vimps.max()
vimps/=norm

In [ ]:
# LINEAR SYSTEM
n=7 # Number of points to reconstruct spline

print("Size of linear system:",4*n)
N=4*n-1 #Last element of matrix

L=n-3+2 # Number of required moments
print("Number of required moments:",L)

# GET IMPACT VELOCITIES
ptot=0
mu=np.zeros((L,1))
vsel=[]
for i in range(len(vimps)):
    vimp=vimps[i]
    #if vimp>40/norm:continue
    vsel+=[vimp]
    prob=pprob[i]
    for k in range(L):
        mu[k]+=vimp**k*prob
    ptot+=prob
vimps=np.array(vsel)
vmin=vimps.min()
vmax=vimps.max()

mu/=ptot

print("Total probability:",ptot)
print("Velocity moments:",mu*norm)
print("Velocity range:",vmin,vmax)

In [ ]:
x=np.linspace(vmin,vmax,n+1) # Sampling points
#"""
dv=0.5*1.0/norm
v1=vmin
v2=vmin+dv
v3=vmin+2*dv
v4=vmax-10*dv
print(v1,v2,v3,v4)
x=np.array([v1,v2])
x=np.concatenate((x,np.linspace(v3,v4,n-2)))
x=np.concatenate((x,[vmax]))
#"""
print("Sampling points:",x)

In [ ]:
def fspline(x,xs,s):
    if x==xs[0] or x==xs[-1]:return 0
    i=np.arange(len(xs))[(x-xs)>=0][-1]
    f=s[4*i]+s[4*i+1]*(x-xs[i])+s[4*i+2]*(x-xs[i])**2+s[4*i+3]*(x-xs[i])**3
    return f

In [ ]:
M=np.zeros((4*n,4*n))
b=np.zeros((4*n,1))

e=0
# Eq. (28)
M[e,0]=1;e+=1 #
M[e,2]=1;e+=1 # Positive steep at x = 0
M[e,3]=1;e+=1 #

# Eq. (30)
M[e,N-3:]=[1,(x[n]-x[n-1]),(x[n]-x[n-1])**2,(x[n]-x[n-1])**3];e+=1 #

# Eq. (31)
for i in range(n-1):
    M[e,4*i:4*i+5]=[1,(x[i+1]-x[i]),(x[i+1]-x[i])**2,(x[i+1]-x[i])**3,-1];e+=1 #

# Eq. (32)
for i in range(n-1):
    M[e,4*i+1:4*i+1+3]=[1,2*(x[i+1]-x[i]),3*(x[i+1]-x[i])**2];
    M[e,4*(i+1)+1:4*(i+1)+1+1]=[-1]
    e+=1 

# Eq. (33)
for i in range(n-1):
    M[e,4*i+2:4*i+2+2]=[2*(x[i+1]-x[i]),6*(x[i+1]-x[i])]
    M[e,4*(i+1)+2:4*(i+1)+2+1]=[-1]
    e+=1 

# MOMENTS
for k in range(L):
    m=0
    for i in range(n):
        I1=(x[i+1]**(k+1)-x[i]**(k+1))/(k+1)
        I2=(x[i+1]**(k+2)-x[i]**(k+2))/(k+2)
        I3=(x[i+1]**(k+3)-x[i]**(k+3))/(k+3)
        I4=(x[i+1]**(k+4)-x[i]**(k+4))/(k+4)
        # print i,I1,I2,I3,I4
        M[e,m]=I1;m+=1
        M[e,m]=(I2-x[i]*I1);m+=1
        M[e,m]=(I3-2*x[i]*I2+x[i]**2*I1);m+=1
        M[e,m]=(I4-3*x[i]*I3+3*x[i]**2*I2-x[i]**3*I1);m+=1
    b[e]=mu[k]
    e+=1 

# SOLVE
s=np.linalg.solve(M,b)
#print "Coefficients:",s

# SPLINE FUNCTION
vs=np.linspace(vmin,vmax,4*n)
ps=np.array([fspline(v,x,s) for v in vs])

print("Velocities:",vs)
print("Probabilities:",ps)

In [ ]:
# TEST MOMENTS
from scipy.interpolate import interp1d as interp
from scipy.integrate import quad as integrate
pinterp=interp(vs,ps,kind='slinear')

print("Reconstructed function moments:")
for k in range(L):
    func=lambda x:x**k*pinterp(x)
    i,e=integrate(func,vmin,vmax)
    print("\tMoment %d:"%k,i*norm)
print("Velocity moments:",mu*norm)

In [ ]:
fig=plt.figure()
ax=fig.gca()
ax.plot(vs,ps,'ko-')
ax.hist(sporadic["V inf"]/norm,bins=30,normed=True)
xts=ax.get_xticks()
xtls=[]
for xt in xts:
    xtls+=['%.1f'%(norm*xt)]
ax.set_xticklabels(xtls)

In [ ]:
fig=plt.figure()
ax=fig.gca()
ax.plot(vs,ps,'ko-')
xts=ax.get_xticks()
xtls=[]
for xt in xts:
    xtls+=['%.1f'%(norm*xt)]
ax.set_xticklabels(xtls)

# SPLINE FUNCTION
vs=np.linspace(vmin,vmax,4*n)
ps=np.array([fspline(v,x,s) for v in vs])

print "Velocities:",vs
print "Probabilities:",ps

# TEST MOMENTS
from scipy.interpolate import interp1d as interp
from scipy.integrate import quad as integrate
pinterp=interp(vs,ps,kind='slinear')

print "Reconstructed function moments:"
for k in xrange(L):
    func=lambda x:x**k*pinterp(x)
    i,e=integrate(func,vmin,vmax)
    print "\tMoment %d:"%k,i*norm

fig=plt.figure()
ax=fig.gca()
ax.plot(vs,ps,'ko-')
xts=ax.get_xticks()
xtls=[]
for xt in xts:
    xtls+=['%.1f'%(norm*xt)]
ax.set_xticklabels(xtls)

np.savetxt(FIGDIR+"vreconstructed.txt",np.vstack((100*vs,ps)).transpose())
fig.savefig(FIGDIR+"vreconstructed.png")
return